## Overview

Many climate and meteorological datasets come as gridded rasters in data formats such as NetCDF and GRIB. We will use [XArray](http://xarray.pydata.org/) to read, process and visualize the gridded raster dataset.

Xarray is an evolution of rasterio and is inspired by libraries like pandas to work with raster datasets. It is particularly suited for working with multi-dimensional time-series raster datasets. It also integrates tightly with dask that allows one to scale raster data processing using parallel computing. XArray provides [Plotting Functions](https://xarray.pydata.org/en/stable/user-guide/plotting.html) based on Matplotlib.

In this section, we will take the [Gridded Monthly Temperature Anomaly Data](https://data.giss.nasa.gov/gistemp/) from 1880-present from GISTEMP and visualize the temperature anomaly for the year 2021.

## Setup and Data Download

The following blocks of code will install the required packages and download the datasets to your Colab environment.

In [44]:
import os
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np

In [2]:
data_folder = 'data'
output_folder = 'output'

if not os.path.exists(data_folder):
    os.mkdir(data_folder)
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

In [3]:
def download(url):
    filename = os.path.join(data_folder, os.path.basename(url))
    if not os.path.exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

filename = 'gistemp1200_GHCNv4_ERSSTv5.nc'
data_url = 'https://github.com/spatialthoughts/python-dataviz-web/raw/main/data/gistemp/'

download(data_url + filename)

Downloaded data/gistemp1200_GHCNv4_ERSSTv5.nc


## XArray Terminology

By convention, XArray is imported as `xr`. We use Xarray's `open_dataset()` method to read the gridded raster. The result is a `xarray.Dataset` object.


In [4]:
file_path = os.path.join(data_folder, filename)
ds = xr.open_dataset(file_path)
ds

<xarray.Dataset>
Dimensions:      (lat: 90, lon: 180, time: 1704, nv: 2)
Coordinates:
  * lat          (lat) float32 -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
  * lon          (lon) float32 -179.0 -177.0 -175.0 -173.0 ... 175.0 177.0 179.0
  * time         (time) datetime64[ns] 1880-01-15 1880-02-15 ... 2021-12-15
Dimensions without coordinates: nv
Data variables:
    time_bnds    (time, nv) datetime64[ns] ...
    tempanomaly  (time, lat, lon) float32 ...
Attributes:
    title:        GISTEMP Surface Temperature Analysis
    institution:  NASA Goddard Institute for Space Studies
    source:       http://data.giss.nasa.gov/gistemp/
    Conventions:  CF-1.6
    history:      Created 2022-01-11 09:09:58 by SBBX_to_nc 2.0 - ILAND=1200,...

The NetCDF file contains a grid of values for each month from 1880-2021 at a spatial resolution of 2 degrees. Let's understand what is contained in a Dataset.

* *Variables*: This is similar to a band in a raster dataset. Each variable contains an array of values.
* *Dimensions*: This is similar to number of array axes.
* *Coordinates*: These are the labels for values in each dimension.
* *Attributes*: This is the metadata associated with the dataset.

<img src='https://courses.spatialthoughts.com/images/python_dataviz/xarray_terminology.png' width=800/>

A Dataset consists of one or more `xarray.DataArray` object. This is the main object that consists of a single variable with dimension names, coordinates and attributes. You can access each variable using `dataset.variable_name` syntax.

Let's see the `time_bnds` variable. This contains a 2d array which has both a starting and ending time for each one averaging period.

In [5]:
ds.time_bnds

<xarray.DataArray 'time_bnds' (time: 1704, nv: 2)>
[3408 values with dtype=datetime64[ns]]
Coordinates:
  * time     (time) datetime64[ns] 1880-01-15 1880-02-15 ... 2021-12-15
Dimensions without coordinates: nv

The main variable of interest is = `tempanomaly` - containing the grid of temperature anomaly values at different times. Let's select that variable and store it as `da`.

In [6]:
da = ds.tempanomaly
da

<xarray.DataArray 'tempanomaly' (time: 1704, lat: 90, lon: 180)>
[27604800 values with dtype=float32]
Coordinates:
  * lat      (lat) float32 -89.0 -87.0 -85.0 -83.0 -81.0 ... 83.0 85.0 87.0 89.0
  * lon      (lon) float32 -179.0 -177.0 -175.0 -173.0 ... 175.0 177.0 179.0
  * time     (time) datetime64[ns] 1880-01-15 1880-02-15 ... 2021-12-15
Attributes:
    long_name:     Surface temperature anomaly
    units:         K
    cell_methods:  time: mean

## Selecting Data

XArray provides a very powerful way to select subsets of data, using similar framework as Pandas. Similar to Panda's `loc` and `iloc` methods, XArray provides `sel` and `isel` methods. Since DataArray dimensions have names, these methods allow you to specify which dimension to query.

Let's select the temperature anomany values for the last time step. Since we know the index (-1) of the data we can use `isel` method.

In [7]:
da.isel(time=-1)

<xarray.DataArray 'tempanomaly' (lat: 90, lon: 180)>
[16200 values with dtype=float32]
Coordinates:
  * lat      (lat) float32 -89.0 -87.0 -85.0 -83.0 -81.0 ... 83.0 85.0 87.0 89.0
  * lon      (lon) float32 -179.0 -177.0 -175.0 -173.0 ... 175.0 177.0 179.0
    time     datetime64[ns] 2021-12-15
Attributes:
    long_name:     Surface temperature anomaly
    units:         K
    cell_methods:  time: mean

We can also specify a value to query using the `sel()` method.

In [8]:
da.sel(time='2021-12-15')

<xarray.DataArray 'tempanomaly' (lat: 90, lon: 180)>
[16200 values with dtype=float32]
Coordinates:
  * lat      (lat) float32 -89.0 -87.0 -85.0 -83.0 -81.0 ... 83.0 85.0 87.0 89.0
  * lon      (lon) float32 -179.0 -177.0 -175.0 -173.0 ... 175.0 177.0 179.0
    time     datetime64[ns] 2021-12-15
Attributes:
    long_name:     Surface temperature anomaly
    units:         K
    cell_methods:  time: mean

We can specify multiple dimensions to query for a subset. Let's extract the temperature anomaly at `lat=49`, `lon=-123` and `time='2021-06-15'`. This region experienced abnormally high temperatures in June 2021.

In [9]:
da.sel(lat=49, lon=-123, time='2021-06-15')

<xarray.DataArray 'tempanomaly' ()>
[1 values with dtype=float32]
Coordinates:
    lat      float32 49.0
    lon      float32 -123.0
    time     datetime64[ns] 2021-06-15
Attributes:
    long_name:     Surface temperature anomaly
    units:         K
    cell_methods:  time: mean

The `sel()` method also support nearest neighbor lookups. This is useful when you do not know the exact label of the dimension, but want to find the closest one.

> Tip: You can use `interp()` instead of `sel()` to interpolate the value instead of closest lookup.

In [10]:
# find nearest neighbour to specified coordinates & time
da.sel(lat=28.6, lon=77.2, time='2021-05-01', method='nearest')

<xarray.DataArray 'tempanomaly' ()>
[1 values with dtype=float32]
Coordinates:
    lat      float32 29.0
    lon      float32 77.0
    time     datetime64[ns] 2021-05-15
Attributes:
    long_name:     Surface temperature anomaly
    units:         K
    cell_methods:  time: mean

You can call `.values` on a DataArray to get an array of the values.

In [11]:
# get values of nearest neighbour to specified coordinates & time
selected = da.sel(lat=28.6, lon=77.2, time='2021-05-01', method='nearest')
print(selected.values)

-0.76


The `sel()` method also allows specifying range of values using Python's built-in `slice()` function. The code below will select all observationss in the year 2021.

In [ ]:
# select ALL observations in 2021
da.sel(time=slice('2021-01-01', '2021-12-31'))

## Masking and Subsetting Data

XArray has a [`where()`](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.where.html) function that allows you to extract a subset of the array. The code block below extracts the anomaly at a specific Lat/Lon. We then use the `.where()` function to select items that have a positive anomaly.


In [13]:
# extract a subset of the array
selected = da.sel(lat=28.6, lon=77.2, method='nearest')
selected

<xarray.DataArray 'tempanomaly' (time: 1704)>
[1704 values with dtype=float32]
Coordinates:
    lat      float32 29.0
    lon      float32 77.0
  * time     (time) datetime64[ns] 1880-01-15 1880-02-15 ... 2021-12-15
Attributes:
    long_name:     Surface temperature anomaly
    units:         K
    cell_methods:  time: mean

We can use `drop=True` to remove all items where the condition did not match and create a subset of the data.

In [14]:
# select only positive anomalies 
positive = selected.where(selected > 0, drop=True) # drop true removes items where condition not met
positive

<xarray.DataArray 'tempanomaly' (time: 798)>
array([0.38      , 2.76      , 1.54      , 0.06      , 1.15      ,
       0.07      , 0.79999995, 0.45      , 0.45999998, 0.72999996,
       0.83      , 0.41      , 0.26      , 0.09      , 0.24      ,
       0.93      , 0.93      , 0.35      , 0.68      , 0.85999995,
       0.69      , 0.02      , 0.94      , 0.32999998, 0.28      ,
       0.16      , 0.22      , 1.0799999 , 1.14      , 0.31      ,
       1.3399999 , 2.74      , 0.56      , 0.28      , 1.41      ,
       0.04      , 0.66999996, 0.53      , 0.38      , 0.84      ,
       1.23      , 0.42      , 0.09      , 0.09      , 0.95      ,
       0.59      , 0.94      , 1.8299999 , 0.26999998, 0.09999999,
       1.12      , 0.76      , 1.15      , 0.95      , 0.13      ,
       0.37      , 0.87      , 1.38      , 0.72999996, 0.88      ,
       0.09999999, 0.59999996, 0.72999996, 0.76      , 0.79999995,
       0.14      , 1.13      , 0.58      , 1.4       , 0.24      ,
       0.35999998, 0.08      , 0.51      , 0.89      , 1.2099999 ,
       0.08      , 0.48      , 0.87      , 0.61      , 0.22      ,
       0.68      , 0.06      , 0.42999998, 1.51      , 0.14999999,
       0.57      , 0.45      , 1.51      , 1.3199999 , 0.09999999,
       0.48999998, 0.74      , 0.04      , 0.48999998, 0.51      ,
       0.16      , 0.68      , 0.97999996, 1.4       , 0.71      ,
...
       0.39      , 0.38      , 0.89      , 1.35      , 1.15      ,
       0.32999998, 0.21      , 0.04      , 0.5       , 0.76      ,
       0.52      , 1.01      , 0.19      , 1.31      , 0.29999998,
       0.14      , 1.01      , 1.05      , 0.22      , 0.75      ,
       0.21      , 1.25      , 1.11      , 1.0699999 , 0.52      ,
       0.87      , 1.17      , 1.8199999 , 0.04      , 1.12      ,
       0.12      , 0.65999997, 1.0799999 , 1.42      , 1.75      ,
       0.85999995, 1.5799999 , 1.8       , 1.76      , 1.88      ,
       1.4499999 , 0.98999995, 0.35      , 0.32999998, 1.22      ,
       1.38      , 1.14      , 2.29      , 0.72999996, 1.75      ,
       0.90999997, 2.07      , 1.43      , 0.29999998, 0.82      ,
       0.97999996, 1.8       , 0.62      , 1.13      , 1.35      ,
       2.2       , 2.56      , 1.56      , 1.31      , 0.53999996,
       0.53999996, 0.82      , 0.35999998, 0.55      , 1.27      ,
       0.17      , 0.35999998, 1.54      , 0.47      , 1.06      ,
       0.96      , 0.75      , 1.22      , 0.42      , 1.14      ,
       0.81      , 0.09999999, 0.11      , 1.2099999 , 1.05      ,
       1.49      , 1.05      , 0.35      , 0.57      , 2.46      ,
       2.4199998 , 0.68      , 1.09      , 1.01      , 0.94      ,
       0.90999997, 0.62      , 0.28      ], dtype=float32)
Coordinates:
    lat      float32 29.0
    lon      float32 77.0
  * time     (time) datetime64[ns] 1880-01-15 1880-03-15 ... 2021-12-15
Attributes:
    long_name:     Surface temperature anomaly
    units:         K
    cell_methods:  time: mean

## Aggregating Data

A very-powerful feature of XArray is the ability to easily aggregate data across dimensions - making it ideal for many remote sensing analysis. Let's calculate the average temperature anomany for the year 2021.

We first select the subset for year 2021 and apply the `.mean()` aggregation across the `time` dimension.

In [15]:
# subset 2021 data
subset2021 = da.sel(time=slice('2021-01-01', '2021-12-31'))

# calculate aavergae temp anomaly
subset2021.mean(dim='time')

<xarray.DataArray 'tempanomaly' (lat: 90, lon: 180)>
array([[-1.0675001, -1.0675001, -1.0675001, ..., -1.0675001, -1.0675001,
        -1.0675001],
       [-1.0675001, -1.0675001, -1.0675001, ..., -1.0675001, -1.0675001,
        -1.0675001],
       [-1.0675001, -1.0675001, -1.0675001, ..., -1.0675001, -1.0675001,
        -1.0675001],
       ...,
       [ 3.7291667,  3.7291667,  3.7291667, ...,  3.7291667,  3.7291667,
         3.7291667],
       [ 3.7291667,  3.7291667,  3.7291667, ...,  3.7291667,  3.7291667,
         3.7291667],
       [ 3.7291667,  3.7291667,  3.7291667, ...,  3.7291667,  3.7291667,
         3.7291667]], dtype=float32)
Coordinates:
  * lat      (lat) float32 -89.0 -87.0 -85.0 -83.0 -81.0 ... 83.0 85.0 87.0 89.0
  * lon      (lon) float32 -179.0 -177.0 -175.0 -173.0 ... 175.0 177.0 179.0

XArray has many features that easily work with time-series data such as this. We can use temporal components to aggregate the data across time. Here we take our monthly time-series of anomalies and aggregate it to a yearly time-series using the `groupby()` method.

Reference: [Resampling and grouped operations](https://docs.xarray.dev/en/stable/user-guide/time-series.html#resampling-and-grouped-operations)

In [16]:
yearly = da.groupby('time.year').mean(dim='time')
yearly

<xarray.DataArray 'tempanomaly' (year: 142, lat: 90, lon: 180)>
array([[[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
...
        [ 3.7966664 ,  3.7966664 ,  3.7966664 , ...,  3.7966664 ,
          3.7966664 ,  3.7966664 ],
        [ 3.7966664 ,  3.7966664 ,  3.7966664 , ...,  3.7966664 ,
          3.7966664 ,  3.7966664 ],
        [ 3.7966664 ,  3.7966664 ,  3.7966664 , ...,  3.7966664 ,
          3.7966664 ,  3.7966664 ]],

       [[-1.0675001 , -1.0675001 , -1.0675001 , ..., -1.0675001 ,
         -1.0675001 , -1.0675001 ],
        [-1.0675001 , -1.0675001 , -1.0675001 , ..., -1.0675001 ,
         -1.0675001 , -1.0675001 ],
        [-1.0675001 , -1.0675001 , -1.0675001 , ..., -1.0675001 ,
         -1.0675001 , -1.0675001 ],
        ...,
        [ 3.7291667 ,  3.7291667 ,  3.7291667 , ...,  3.7291667 ,
          3.7291667 ,  3.7291667 ],
        [ 3.7291667 ,  3.7291667 ,  3.7291667 , ...,  3.7291667 ,
          3.7291667 ,  3.7291667 ],
        [ 3.7291667 ,  3.7291667 ,  3.7291667 , ...,  3.7291667 ,
          3.7291667 ,  3.7291667 ]]], dtype=float32)
Coordinates:
  * lat      (lat) float32 -89.0 -87.0 -85.0 -83.0 -81.0 ... 83.0 85.0 87.0 89.0
  * lon      (lon) float32 -179.0 -177.0 -175.0 -173.0 ... 175.0 177.0 179.0
  * year     (year) int64 1880 1881 1882 1883 1884 ... 2017 2018 2019 2020 2021
Attributes:
    long_name:     Surface temperature anomaly
    units:         K
    cell_methods:  time: mean

## Exercise

Can you find out when the highest temperature anomaly occured at a specific location?

Replace the `lat` and `lon` in the following code with your chosen location. You will see the resulting `max_anomaly` DataArray with the anomaly value along with lat, lon and time coordinates.  Extract the `time` coordinate of the resulting array and print the time when the maximum anomaly occured.

In [17]:
selected = da.sel(lat=28.6, lon=77.2, method='nearest')
max_anomaly = selected.where(selected==selected.max(), drop=True)
max_anomaly

<xarray.DataArray 'tempanomaly' (time: 1)>
array([5.98], dtype=float32)
Coordinates:
    lat      float32 51.0
    lon      float32 19.0
  * time     (time) datetime64[ns] 1990-02-15
Attributes:
    long_name:     Surface temperature anomaly
    units:         K
    cell_methods:  time: mean

## My Solution

In [18]:
# Krakow 
selected = da.sel(lat=50.0647, lon=19.9450, method='nearest')
max_anomaly = selected.where(selected==selected.max(), drop=True)
max_anomaly

<xarray.DataArray 'tempanomaly' (time: 1)>
array([5.98], dtype=float32)
Coordinates:
    lat      float32 51.0
    lon      float32 19.0
  * time     (time) datetime64[ns] 1990-02-15
Attributes:
    long_name:     Surface temperature anomaly
    units:         K
    cell_methods:  time: mean

In [47]:
# obtain the value of the time cooordinate
krakow_max = max_anomaly.time.values
krakow_max

array(['1990-02-15T00:00:00.000000000'], dtype='datetime64[ns]')

In [53]:
# to make printable i.e remove the square brackets, convert to list 
date = krakow_max.tolist()[0]

# convert back to datetime64
dt64 = np.datetime64(date, 'ns')

In [54]:
print(f'The maximum temperature anomaly in Krakow ocurred on {dt64}')

The maximum temperature anomaly in Krakow ocurred on 1990-02-15T00:00:00.000000000
